In [ ]:
home_dir = "./drive/MyDrive/趣味/OhtaniSystem"
data_dir = home_dir + "/data"
model_dir = home_dir + "/model"
test_dir = data_dir + "/test"
script_dir = home_dir + "/script"

分類モデル作成パート

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
!pip install icrawler
!pip install IPython

In [ ]:
from icrawler.builtin import BingImageCrawler

crawler = BingImageCrawler(storage={"root_dir": data_dir + "/ohtani"})
crawler.crawl(keyword="大谷翔平",max_num=100)

2021-07-11 12:49:46,861 - INFO - icrawler.crawler - start crawling...
2021-07-11 12:49:46,864 - INFO - icrawler.crawler - starting 1 feeder threads...
2021-07-11 12:49:46,865 - INFO - feeder - thread feeder-001 exit
2021-07-11 12:49:46,869 - INFO - icrawler.crawler - starting 1 parser threads...
2021-07-11 12:49:46,872 - INFO - icrawler.crawler - starting 1 downloader threads...
2021-07-11 12:49:47,207 - INFO - parser - parsing result page https://www.bing.com/images/async?q=大谷翔平&first=0
2021-07-11 12:49:48,093 - INFO - downloader - image #1	https://the-ans.jp/wp-content/uploads/2018/05/20180521_Shohei-Otani.jpg
2021-07-11 12:49:48,929 - INFO - downloader - image #2	http://ripples-of-caret.com/wp-content/uploads/2018/04/8d193aa6376afa043f18c8c185b11268.jpg
2021-07-11 12:49:49,148 - INFO - downloader - image #3	https://www.sankei.com/images/news/181113/wor1811130006-p1.jpg
2021-07-11 12:49:51,967 - INFO - downloader - image #4	https://images.daznservices.com/di/library/sporting_news/98/

In [ ]:
crawler = BingImageCrawler(storage={"root_dir": data_dir + "/Youtuber"})
crawler.crawl(keyword="Youtuber",max_num=100)

2021-07-11 12:50:44,965 - INFO - icrawler.crawler - start crawling...
2021-07-11 12:50:44,966 - INFO - icrawler.crawler - starting 1 feeder threads...
2021-07-11 12:50:44,970 - INFO - feeder - thread feeder-001 exit
2021-07-11 12:50:44,971 - INFO - icrawler.crawler - starting 1 parser threads...
2021-07-11 12:50:44,978 - INFO - icrawler.crawler - starting 1 downloader threads...
2021-07-11 12:50:45,328 - INFO - parser - parsing result page https://www.bing.com/images/async?q=Youtuber&first=0
2021-07-11 12:50:47,016 - INFO - downloader - image #1	http://www.culturenorthernireland.org/sites/default/files/content/Creativity Month 2018.jpg
2021-07-11 12:50:47,075 - INFO - downloader - image #2	https://i.ytimg.com/vi/nmTg05XG-TE/maxresdefault.jpg
2021-07-11 12:50:47,122 - INFO - downloader - image #3	https://i.ytimg.com/vi/uzl_WBllhJI/maxresdefault.jpg
2021-07-11 12:50:47,139 - INFO - downloader - image #4	https://i.ytimg.com/vi/09U0JCDoJHs/maxresdefault.jpg
2021-07-11 12:50:47,767 - INFO -

In [ ]:
from PIL import Image
import os, glob
import numpy as np
from PIL import ImageFile
# IOError: image file is truncated (0 bytes not processed)回避のため
ImageFile.LOAD_TRUNCATED_IMAGES = True

classes = ["ohtani", "Youtuber"]
num_classes = len(classes)
image_size = 64
num_testdata = 25

X_train = []
X_test  = []
y_train = []
y_test  = []

for index, classlabel in enumerate(classes):
    photos_dir = data_dir + "/" + classlabel
    files = glob.glob(photos_dir + "/*.jpg")
    for i, file in enumerate(files):
        image = Image.open(file)
        image = image.convert("RGB")
        image = image.resize((image_size, image_size))
        data = np.asarray(image)
        if i < num_testdata:
            X_test.append(data)
            y_test.append(index)
        else:

            # angleに代入される値
            # -20
            # -15
            # -10
            #  -5
            # 0
            # 5
            # 10
            # 15
            # 画像を5度ずつ回転
            for angle in range(-20, 20, 5):

                img_r = image.rotate(angle)
                data = np.asarray(img_r)
                X_train.append(data)
                y_train.append(index)
                # FLIP_LEFT_RIGHT　は 左右反転
                img_trains = img_r.transpose(Image.FLIP_LEFT_RIGHT)
                data = np.asarray(img_trains)
                X_train.append(data)
                y_train.append(index)

X_train = np.array(X_train)
X_test  = np.array(X_test)
y_train = np.array(y_train)
y_test  = np.array(y_test)

xy = (X_train, X_test, y_train, y_test)
np.save(data_dir + "/ohtani_youtuber.npy", xy)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import RMSprop
from keras.utils import np_utils
import keras
import numpy as np

classes = ["ohtani", "Youtuber"]
num_classes = len(classes)
image_size = 64

"""
データを読み込む関数
"""
def load_data():
    X_train, X_test, y_train, y_test = np.load(data_dir + "/ohtani_youtuber.npy", allow_pickle=True)
    # 入力データの各画素値を0-1の範囲で正規化(学習コストを下げるため)
    X_train = X_train.astype("float") / 255
    X_test  = X_test.astype("float") / 255
    # to_categorical()にてラベルをone hot vector化
    y_train = np_utils.to_categorical(y_train, num_classes)
    y_test  = np_utils.to_categorical(y_test, num_classes)

    return X_train, y_train, X_test, y_test

"""
モデルを学習する関数
"""
def train(X, y, X_test, y_test):
    model = Sequential()

    # Xは(1200, 64, 64, 3)
    # X.shape[1:]とすることで、(64, 64, 3)となり、入力にすることが可能です。
    model.add(Conv2D(32,(3,3), padding='same',input_shape=X.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(64,(3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.45))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    # https://keras.io/ja/optimizers/
    # 今回は、最適化アルゴリズムにRMSpropを利用
    opt = RMSprop(lr=0.00005, decay=1e-6)
    # https://keras.io/ja/models/sequential/
    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    model.fit(X, y, batch_size=28, epochs=40)
    # HDF5ファイルにKerasのモデルを保存
    model.save(model_dir + '/cnn.h5')

    return model

"""
メイン関数
データの読み込みとモデルの学習を行います。
"""
def main():
    # データの読み込み
    X_train, y_train, X_test, y_test = load_data()

    # モデルの学習
    model = train(X_train, y_train, X_test, y_test)

main()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/40
72/72 [==============================] - 45s 12ms/step - loss: 0.6670 - accuracy: 0.6113
Epoch 2/40
72/72 [==============================] - 1s 12ms/step - loss: 0.5797 - accuracy: 0.7158
Epoch 3/40
72/72 [==============================] - 1s 12ms/step - loss: 0.5240 - accuracy: 0.7535
Epoch 4/40
72/72 [==============================] - 1s 12ms/step - loss: 0.4417 - accuracy: 0.8217
Epoch 5/40
72/72 [==============================] - 1s 12ms/step - loss: 0.3628 - accuracy: 0.8511
Epoch 6/40
72/72 [==============================] - 1s 12ms/step - loss: 0.2742 - accuracy: 0.8874
Epoch 7/40
72/72 [==============================] - 1s 12ms/step - loss: 0.2163 - accuracy: 0.9204
Epoch 8/40
72/72 [==============================] - 1s 11ms/step - loss: 0.1666 - accuracy: 0.9441
Epoch 9/40
72/72 [==============================] - 1s 11ms/step - loss: 0.1409 - accuracy: 0.9507
Epoch 10/40
72/72 [==============================] - 1s 12ms/step - loss: 0.1050 - accuracy: 0.9661
Epoch 11

In [ ]:
import keras
import sys, os
import numpy as np
from keras.models import load_model

imsize = (64, 64)

"""
dog1.jpgというファイル名の画像をGoogle Colab上にアップロードする方法は2通りあります。
1つが、下記のコードを実行し画像をアップロードする方法
from google.colab import files
uploaded = files.upload()
2つが、Colab左メニューの>アイコンを押して、目次、コード スニペット、ファイル
の3つ表示されるますが、右のファイルタブから画像をアップロードする方法です。
このファイルタブをクリックするとアップロードと更新の2つがありますが、
アップロードを押すと画像をアップロードすることが可能です。
"""

#testpic     = test_dir + "/ohtani.jpg"
testpic     = test_dir + "/hikakin.png"
keras_param = model_dir + "/cnn.h5"

def load_image(path):
    img = Image.open(path)
    img = img.convert('RGB')
    # 学習時に、(64, 64, 3)で学習したので、画像の縦・横は今回 変数imsizeの(64, 64)にリサイズします。
    img = img.resize(imsize)
    # 画像データをnumpy配列の形式に変更
    img = np.asarray(img)
    img = img / 255.0
    return img

model = load_model(keras_param)
img = load_image(testpic)
prd = model.predict(np.array([img]))
print(prd) # 精度の表示
prelabel = np.argmax(prd, axis=1)
if prelabel == 0:
    print(">>> 大谷翔平")
elif prelabel == 1:
    print(">>> それ以外")

[[1.6855087e-16 1.0000000e+00]]
>>> それ以外


h5 to tflite パート

In [ ]:
import tensorflow as tf
try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow.keras import layers

print("TF version:", tf.__version__)

TF version: 2.5.0


In [ ]:
from tensorflow import keras
model = keras.models.load_model(model_dir + "/cnn.h5")

In [ ]:
# モデルを変換
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model_keras = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpvumxvxzn/assets


2021-07-11 13:09:58,085 - INFO - tensorflow - Assets written to: /tmp/tmpvumxvxzn/assets


In [ ]:
open(model_dir + '/ohtani.tflite', 'wb').write(tflite_model_keras)

25963064

メタデータ追加パート(AddMetaData.pyを編集して利用)

In [ ]:
#export directoryは天邪鬼．良いときと悪いときの違いが判らん.適当なディレクトリを指定して通るまで試す

In [ ]:
print(script_dir)

./drive/MyDrive/趣味/OhtaniSystem/script


In [ ]:
!pip install tflite_support

     |████████████████████████████████| 1.1MB 7.4MB/s 
     |████████████████████████████████| 194kB 25.7MB/s 


In [40]:
!python ./drive/MyDrive/趣味/OhtaniSystem/script/AddMetaData.py \
    --model_file=./drive/MyDrive/趣味/OhtaniSystem/model/ohtani.tflite \
    --label_file=./drive/MyDrive/趣味/OhtaniSystem/model/label.txt \
    --export_directory=./drive/MyDrive/data

2021-07-11 13:53:46.109318: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
実行
Finished populating metadata and associated file to the model:
./drive/MyDrive/趣味/OhtaniSystem/model/ohtani.tflite
The metadata json file has been saved to:
./drive/MyDrive/data/ohtani.json
The associated file that has been been packed to the model is:
['label.txt']
